In [1]:
import numpy as np
import pickle as pkl
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("cleaned_ECG.csv")
df.drop(columns=['ECG2'],inplace=True) #considering only one ECG signal for forecasting
df.columns = ['ds', 'y']
df.shape

(7680, 2)

In [3]:
quantiles = [0.015, 0.985]

params = {
    "n_lags": 24,
    "n_forecasts": 7,
    "n_changepoints": 20,
    "learning_rate": 0.01,
    "ar_layers": [32, 16, 16, 32],
    "epochs": 70,
    "batch_size": 64,
    "quantiles": quantiles,
}


m = NeuralProphet(**params)
m.set_plotting_backend("plotly-static")
set_log_level("ERROR")

In [25]:
df_train, df_test = m.split_df(df, valid_p=0.05, local_split=True)
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (7292, 2)
Test shape: (412, 2)


In [26]:
from tensorflow.keras.models import load_model
lstm_model = load_model("model_store/lstm_ecg.keras")
lstm_model.summary()

with open("model_store/best_order_ecg.pkl", "rb") as f:
    loaded_order = pkl.load(f)

with open("model_store/opt_no_states_ecg.pkl", "rb") as f:
    opt_states = pkl.load(f)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                23100     
                                                                 
 dense (Dense)               (None, 1)                 76        
                                                                 
Total params: 23176 (90.53 KB)
Trainable params: 23176 (90.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
from hmmlearn import hmm
from statsmodels.tsa.arima.model import ARIMA
from utils import softmax_weighting, get_mae_errors
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(np.array(df['y']).reshape(-1,1))

def dynamic_ensemble_prediction(train, test):
    train_hmm = train.reshape(-1,1)
    train_hmm = scaler.transform(train_hmm)
    test_hmm = test.reshape(-1,1)
    test_hmm = scaler.transform(test_hmm)
    hmm_history = train_hmm
    history = np.array(train)

    predictions = []
    truth_values = []
    lstm_preds = []
    hmm_preds = []
    arima_preds = []

    for i in range(len(test)):
        print(f'{i+1}/{len(test)}')
        truth_values.append(test[i])
        # LSTM
        lstm_pred = lstm_model.predict(history[-24:].reshape(1,24))[0][0]
        lstm_preds.append(lstm_pred)
        # ARIMA
        arima_model = ARIMA(history, order=loaded_order)
        arima_fit = arima_model.fit()
        arima_pred = arima_fit.forecast(steps=1)[0]
        arima_preds.append(arima_pred)
        # HMM
        hmm_model = hmm.GaussianHMM(n_components=opt_states, covariance_type='diag', tol=0.0001, n_iter=100)
        hmm_model.fit(hmm_history)
        hidden_states = hmm_model.predict(hmm_history)
        last_hidden_state = hidden_states[-1]
        next_state_probs = hmm_model.transmat_[last_hidden_state]
        predicted_state = np.argmax(next_state_probs)
        predicted_value = hmm_model.means_[predicted_state][0]
        hmm_pred = scaler.inverse_transform(np.array(predicted_value).reshape(-1,1))[0][0]
        hmm_preds.append(hmm_pred)

        #Error Measurement
        arima_error = get_mae_errors(arima_preds, truth_values)
        hmm_error = get_mae_errors(hmm_preds, truth_values)
        lstm_error = get_mae_errors(lstm_preds, truth_values) 

        weights = softmax_weighting(arima_error, lstm_error, hmm_error) # Weighting algorithm

        predictions.append(weights[0]*arima_pred + weights[1]*lstm_pred + weights[2]*hmm_pred)
        history = np.append(history,test[i])
        
        if i != len(test)-1:
            hmm_history = np.append(hmm_history,test_hmm[i]).reshape(-1,1)
    
    return predictions, arima_preds, hmm_preds, lstm_preds
        

In [ ]:
de_preds, arima_preds, hmm_preds, lstm_preds = dynamic_ensemble_prediction(np.array(df_train.y), np.array(df_test.y))

1/412
1/1 [==============================] - 0s 18ms/step
2/412
1/1 [==============================] - 0s 33ms/step
3/412
1/1 [==============================] - 0s 75ms/step
4/412
1/1 [==============================] - 0s 29ms/step
5/412
1/1 [==============================] - 0s 25ms/step
6/412
1/1 [==============================] - 0s 20ms/step
7/412
1/1 [==============================] - 0s 20ms/step
8/412
1/1 [==============================] - 0s 28ms/step
9/412
1/1 [==============================] - 0s 30ms/step
10/412
1/1 [==============================] - 0s 34ms/step
11/412
1/1 [==============================] - 0s 25ms/step
12/412
1/1 [==============================] - 0s 26ms/step
13/412
1/1 [==============================] - 0s 20ms/step
14/412
1/1 [==============================] - 0s 29ms/step
15/412
1/1 [==============================] - 0s 21ms/step
16/412
1/1 [==============================] - 0s 35ms/step
17/412
1/1 [==============================] - 0s 24ms/step
18/412

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import math
print('Dynamic Ensemble')
print(f'R2 Score : {r2_score(df_test.y,de_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,de_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,de_preds)}')
print('ARIMA')
print(f'R2 Score : {r2_score(df_test.y,arima_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,arima_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,arima_preds)}')
print('HMM')
print(f'R2 Score : {r2_score(df_test.y,hmm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,hmm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,hmm_preds)}')
print('LSTM')
print(f'R2 Score : {r2_score(df_test.y,lstm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y, lstm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,lstm_preds)}')

In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y'], mode='lines', name='Real Data'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_preds, mode='lines', name='Proposed Method'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=arima_preds, mode='lines', name='ARIMA'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=hmm_preds, mode='lines', name='HMM'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=lstm_preds, mode='lines', name='LSTM'))
fig.add_trace(go.Scatter(x=df_train['ds'], y=df_train['y'], mode='lines', name='Training'))
fig.show()